In [ ]:
from langchain.agent initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchaon.chat_model import openAi
from langchaon.tools import tool
import os

In [ ]:
AZURE_KEY = "azure_api_key_here"

@tool
def request_bing(query: str) -> str:
    """
    Searches the internet for additional information if the LLM cannot provide an answer by itself.
    Useful for answering questions about current events or the current state of the world.
    """
    url = "https://api.bing.microsoft.com/v7.0/search"
    headers = {"Ocp-Apim-Subscription-Key": AZURE_KEY}
    params = {"q": query}
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    data = response.json()
    snippets_list = [result['snippet'] for result in data.get('webPages', {}).get('value', [])]
    return "\n".join(snippets_list) if snippets_list else "No relevant results found."

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.2)

tools = [request_bing]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
agent_chain = initialize_agent(
    tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
    verbose=True, memory=memory, handle_parsing_errors=True
)

print("Input something:")
while True:
    query = input()
    if query.lower() == "exit":
        break
    agent_chain.run(input=query)